In [1]:
import pandas as pd
from pathlib import Path 
import os
import glob
from datetime import datetime

main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/structure_donations/"


## Load the annotated data for all platforms

In [2]:

# Folder containing CSVs
folder_annotated = f"{main_path}Annotated_Merged_Structures"

# Dictionary to hold DataFrames
dfs_annotated = {}

for file in os.listdir(folder_annotated):
    if file.endswith(".csv"):
        name = os.path.splitext(file)[0]
        path = os.path.join(folder_annotated, file)
        dfs_annotated[name] = pd.read_csv(path)

In [3]:
dfs_annotated.keys()

dict_keys(['YT_merged_structure_annotated', 'IG_merged_structure_annotated', 'X_merged_structure_annotated', 'TT_merged_structure_annotated', 'YT_merged_column_names_annotated', 'FB_merged_structure_annotated'])

## Loading the (new) merged structures

In [4]:


folder_new = f"{main_path}Processed_structure_donations"

dfs_new = {}

for path in glob.glob(f"{folder_new}/**/Final/*_Merged_structures.csv", recursive=True):
    name = os.path.splitext(os.path.basename(path))[0]
    dfs_new[name] = pd.read_csv(path)


YT_column = os.path.join(folder_new, "Youtube/Column_names_final/YT_Merged_Column_Names.csv")
if os.path.exists(YT_column):
    name = os.path.splitext(os.path.basename(YT_column))[0]
    dfs_new[name] = pd.read_csv(YT_column)

## Identify the new rows

In [5]:


new_rows_dict = {}

dict_dfs = {
    'YT_merged_structure_annotated': 'YT_Merged_structures',
    'IG_merged_structure_annotated': 'IG_Merged_structures',
    'TT_merged_structure_annotated': 'TT_Merged_structures',
    'FB_merged_structure_annotated': 'FB_Merged_structures',
    'X_merged_structure_annotated' : 'X_Merged_structures',
    'YT_merged_column_names_annotated': 'YT_Merged_Column_Names'

}

for k,v in dict_dfs.items():
    
    dfs_annotated[f'{k}'] = dfs_annotated[k].loc[:, ~dfs_annotated[k].columns.str.startswith('Unnamed')]


    merge_cols = list(dfs_annotated[k].columns)

    if 'keepID' in merge_cols:
        merge_cols.remove('keepID')
    if 'keep_id' in merge_cols:
        merge_cols.remove('keep_id')
    if 'date_added' in merge_cols:
        merge_cols.remove('date_added')

   
    new_rows_dict[f'{v}_new'] = dfs_new[v].merge(dfs_annotated[k], how="left", on = merge_cols, indicator=True).query('_merge == "left_only"').drop(columns=["_merge"])

    date_col = 'date_added'

    if date_col not in new_rows_dict[f'{v}_new'].columns:
        new_rows_dict[f'{v}_new'][date_col] = datetime.now().strftime("%Y-%m-%d")


## Append the new rows to the annotated merged structure


In [6]:
for k,v in dict_dfs.items():

    date_col = 'date_added'
    if date_col not in dfs_annotated[f'{k}'].columns:
         dfs_annotated[f'{k}'][date_col] = 'pre-trial'
    
    

    if v == 'YT_Merged_Column_Names':
        merge_cols = ['File_name', 'Column_names']
        use_cols = ['File_name', 'Column_names', 'keepID']

    else: 
        merge_cols = ['json_name', 'path', 'file_path']
        use_cols = ['json_name', 'path', 'file_path', 'keepID']

        file_path = 'file_path'
        if file_path not in dfs_annotated[f'{k}'].columns:
            dfs_annotated[f'{k}'][file_path] = ''
    
    
        
    new_rows_dict[f'{v}_new'] = new_rows_dict[f'{v}_new'].drop('keepID', axis = 1)
    display(new_rows_dict[f'{v}_new'].columns)
    new_rows_dict[f'{v}_new'] = new_rows_dict[f'{v}_new'].merge(dfs_annotated[f'{k}'][use_cols], on = merge_cols, how = 'left')
  
    
    dfs_annotated[f'{k}'] = pd.concat([dfs_annotated[f'{k}'], new_rows_dict[f'{v}_new']], ignore_index=True)
    display(dfs_annotated[f'{k}'])


Index(['json_name', 'id', 'column_name', 'path', 'list_path', 'subfield_path',
       'var_type', 'data_type', 'file_path', 'duplicate_flag', 'date_added'],
      dtype='object')

,json_name,id,column_name,path,list_path,subfield_path,var_type,data_type,file_path,duplicate_flag,keepID,date_added
0,istoricul căutărilor.json,istoricul căutărilor:header,header,"[""header""]",[],"[""header""]",list,string,istoric/istoricul căutărilor.json,No,search-history:header,pre-trial
1,istoricul căutărilor.json,istoricul căutărilor:title,title,"[""title""]",[],"[""title""]",list,string,istoric/istoricul căutărilor.json,No,search-history:title,pre-trial
2,istoricul căutărilor.json,istoricul căutărilor:titleUrl,titleUrl,"[""titleUrl""]",[],"[""titleUrl""]",list,string,istoric/istoricul căutărilor.json,No,search-history:titleUrl,pre-trial
3,istoricul căutărilor.json,istoricul căutărilor:description,description,"[""description""]",[],"[""description""]",list,string,istoric/istoricul căutărilor.json,No,search-history:description,pre-trial
4,istoricul căutărilor.json,istoricul căutărilor:time,time,"[""time""]",[],"[""time""]",list,string,istoric/istoricul căutărilor.json,No,search-history:time,pre-trial
...,...,...,...,...,...,...,...,...,...,...,...,...
123,historial-de-búsqueda.json,historial-de-búsqueda:activityControls,activityControls,"[""activityControls""]",[],"[""activityControls""]",list,string,historial/historial-de-búsqueda.json,No,search-history:activityControls,pre-trial
124,historial-de-reproducciones.json,historial-de-reproducciones:details:name,name,"[""details"", ""name""]","[""details""]","[""name""]",list,string,historial/historial-de-reproducciones.json,No,watch-history:details:name,pre-trial
125,watch-history.json,watch-history:details:name,name,"[""details"", ""name""]","[""details""]","[""name""]",list,string,history/watch-history.json,Yes,watch-history:details:name,pre-trial
126,zoekgeschiedenis.json,zoekgeschiedenis:description,description,"[""description""]",[],"[""description""]",list,string,geschiedenis/zoekgeschiedenis.json,No,search-history:description,pre-trial


Index(['json_name', 'id', 'column_name', 'path', 'list_path', 'subfield_path',
       'var_type', 'data_type', 'file_path', 'duplicate_flag', 'date_added'],
      dtype='object')

,json_name,id,column_name,path,list_path,subfield_path,var_type,data_type,file_path,duplicate_flag,keepID,date_added
0,message_1.json,message_requests:username:message_1:title,title,"[""title""]",NaN,"[""title""]",static,string,your_instagram_activity/messages/message_reque...,No,NaN,pre-trial
1,message_1.json,message_requests:username:message_1:is_still_p...,is_still_participant,"[""is_still_participant""]",NaN,"[""is_still_participant""]",static,boolean,your_instagram_activity/messages/message_reque...,No,NaN,pre-trial
2,message_1.json,message_requests:username:message_1:thread_path,thread_path,"[""thread_path""]",NaN,"[""thread_path""]",static,string,your_instagram_activity/messages/message_reque...,No,NaN,pre-trial
3,message_1.json,message_requests:username:message_1:magic_words,magic_words,"[""magic_words""]",NaN,"[""magic_words""]",static,array,your_instagram_activity/messages/message_reque...,No,NaN,pre-trial
4,message_1.json,username:message_1:is_pending,is_pending,"[""is_pending""]",NaN,"[""is_pending""]",static,boolean,your_instagram_activity/messages/message_reque...,No,NaN,pre-trial
...,...,...,...,...,...,...,...,...,...,...,...,...
676,possible_emails.json,information_about_you:possible_emails:title,title,"[""inferred_data_inferred_emails"", ""title""]","[""inferred_data_inferred_emails""]","[""title""]",list,string,personal_information/information_about_you/pos...,No,NaN,pre-trial
677,possible_emails.json,information_about_you:possible_emails:media_li...,media_list_data,"[""inferred_data_inferred_emails"", ""media_list_...","[""inferred_data_inferred_emails""]","[""media_list_data""]",list,array,personal_information/information_about_you/pos...,No,NaN,pre-trial
678,possible_emails.json,information_about_you:possible_emails:href,href,"[""inferred_data_inferred_emails"", ""string_list...","[""inferred_data_inferred_emails"", ""string_list...","[""href""]",list,string,personal_information/information_about_you/pos...,No,NaN,pre-trial
679,possible_emails.json,information_about_you:possible_emails:value,value,"[""inferred_data_inferred_emails"", ""string_list...","[""inferred_data_inferred_emails"", ""string_list...","[""value""]",list,string,personal_information/information_about_you/pos...,No,NaN,pre-trial


Index(['id', 'column_name', 'path', 'list_path', 'subfield_path', 'var_type',
       'data_type', 'json_name', 'file_path', 'duplicate_flag', 'date_added'],
      dtype='object')

...

,json_name,id,column_name,path,list_path,subfield_path,var_type,data_type,file_path,duplicate_flag,keepID,date_added
0,browser_cookies.json,security_and_login_information:browser_cookies...,YvP_********************,"[""datr_stats_v2"", ""YvP_********************""]",NaN,"[""datr_stats_v2"", ""YvP_********************""]",static,number,security_and_login_information/browser_cookies...,No,NaN,pre-trial
1,message_1.json,your_facebook_activity:messages:filtered_threa...,title,"[""title""]",NaN,"[""title""]",static,string,your_facebook_activity/messages/filtered_threa...,Yes,NaN,pre-trial
2,message_1.json,your_facebook_activity:messages:filtered_threa...,title,"[""title""]",NaN,"[""title""]",static,string,your_facebook_activity/messages/filtered_threa...,Yes,NaN,pre-trial
3,message_1.json,your_facebook_activity:messages:filtered_threa...,title,"[""title""]",NaN,"[""title""]",static,string,your_facebook_activity/messages/filtered_threa...,Yes,NaN,pre-trial
4,message_1.json,your_facebook_activity:messages:filtered_threa...,title,"[""title""]",NaN,"[""title""]",static,string,your_facebook_activity/messages/filtered_threa...,Yes,NaN,pre-trial
...,...,...,...,...,...,...,...,...,...,...,...,...
5686,likes_and_reactions.json,profile_information:your_facebook_activity:com...,timestamp,"[""timestamp""]",NaN,"[""timestamp""]",static,number,personal_information/profile_information/your_...,No,your_facebook_activity:comments_and_reactions:...,NaN
5687,likes_and_reactions.json,profile_information:your_facebook_activity:com...,media,"[""media""]",NaN,"[""media""]",static,array,personal_information/profile_information/your_...,No,your_facebook_activity:comments_and_reactions:...,NaN
5688,likes_and_reactions.json,profile_information:your_facebook_activity:com...,fbid,"[""fbid""]",NaN,"[""fbid""]",static,string,personal_information/profile_information/your_...,No,your_facebook_activity:comments_and_reactions:...,NaN
5689,your_posts__check_ins__photos_and_videos_1.json,profile_information:your_facebook_activity:pos...,timestamp,"[""timestamp""]",NaN,"[""timestamp""]",static,number,personal_information/profile_information/your_...,No,your_facebook_activity:posts:your_posts__check...,NaN


Index(['json_name', 'id', 'column_name', 'path', 'list_path', 'subfield_path',
       'var_type', 'data_type', 'file_path', 'duplicate_flag', 'date_added'],
      dtype='object')

,id,keepID,json_name,column_name,path,list_path,subfield_path,var_type,data_type,file_path,duplicate_flag,date_added
0,manifest:accountId,NaN,manifest.js,accountId,"[""userInfo"", ""accountId""]",NaN,"[""userInfo"", ""accountId""]",static,string,data/manifest.js,No,pre-trial
1,manifest:userName,NaN,manifest.js,userName,"[""userInfo"", ""userName""]",NaN,"[""userInfo"", ""userName""]",static,string,data/manifest.js,No,pre-trial
2,manifest:displayName,NaN,manifest.js,displayName,"[""userInfo"", ""displayName""]",NaN,"[""userInfo"", ""displayName""]",static,string,data/manifest.js,No,pre-trial
3,manifest:sizeBytes,NaN,manifest.js,sizeBytes,"[""archiveInfo"", ""sizeBytes""]",NaN,"[""archiveInfo"", ""sizeBytes""]",static,string,data/manifest.js,No,pre-trial
4,manifest:generationDate,manifest:generationDate,manifest.js,generationDate,"[""archiveInfo"", ""generationDate""]",NaN,"[""archiveInfo"", ""generationDate""]",static,string,data/manifest.js,No,pre-trial
...,...,...,...,...,...,...,...,...,...,...,...,...
629,personalization:advertisers,personalization:advertisers,personalization.js,advertisers,"[""p13nData"", ""interests"", ""audienceAndAdvertis...",NaN,"[""p13nData"", ""interests"", ""audienceAndAdvertis...",static,string,data/personalization.js,Yes,pre-trial
630,personalization:advertisers,personalization:advertisers,personalization.js,advertisers,"[""p13nData"", ""interests"", ""audienceAndAdvertis...",NaN,"[""p13nData"", ""interests"", ""audienceAndAdvertis...",static,array,data/personalization.js,Yes,pre-trial
631,personalization:advertisers,personalization:advertisers,personalization.js,advertisers,"[""p13nData"", ""interests"", ""audienceAndAdvertis...",NaN,"[""p13nData"", ""interests"", ""audienceAndAdvertis...",static,string,data/personalization.js,Yes,pre-trial
632,ip-audit,NaN,ip-audit.js,NaN,NaN,NaN,NaN,skip,NaN,data/ip-audit.js,No,pre-trial


Index(['File_name', 'Column_names', 'id', 'date_added'], dtype='object')

,File_name,Column_names,id,keepID,date_added
0,channel URL configs.csv,Channel ID,channel_URL_configs:Channel_ID,channel_URL_configs:Channel_ID,pre-trial
1,channel URL configs.csv,Channel Vanity URL 1 Name,channel_URL_configs:Channel_Vanity_URL_1_Name,channel_URL_configs:Channel_Vanity_URL_1_Name,pre-trial
2,channel URL configs.csv,Channel Vanity URL 2 Name,channel_URL_configs:Channel_Vanity_URL_2_Name,channel_URL_configs:Channel_Vanity_URL_2_Name,pre-trial
3,channel community moderation settings.csv,Channel ID,channel_community_moderation_settings:Channel_ID,channel_community_moderation_settings:Channel_ID,pre-trial
4,channel feature data.csv,Channel ID,channel_feature_data:Channel_ID,channel_feature_data:Channel_ID,pre-trial
...,...,...,...,...,...
287,Watch later-vídeos.csv,ID de vídeo,Watch_later-vídeos:ID_de_vídeo,Watch_later-videos:Video_ID,pre-trial
288,Watch later-vídeos.csv,Marca de tiempo de creación de la lista de rep...,Watch_later-vídeos:Marca_de_tiempo_de_creació...,Watch_later-videos:Playlist_Video_Creation_Tim...,pre-trial
289,crochet-vídeos.csv,ID de vídeo,crochet-vídeos:ID_de_vídeo,crochet-videos:Video_ID,pre-trial
290,crochet-vídeos.csv,Marca de tiempo de creación de la lista de rep...,crochet-vídeos:Marca_de_tiempo_de_creación_de...,crochet-videos:Playlist_Video_Creation_Timestamp,pre-trial


## Save annotated merged structures

In [7]:


# Assume dfs_dict is your dictionary of DataFrames
for key, df in dfs_annotated.items():
    # Create a filename from the key
    filename = f"{key}.csv"
    path = os.path.join(folder_annotated, filename)
    
    # Save DataFrame to CSV
    df.to_csv(path, index=False)
    print(f"Saved {filename} ({len(df)} rows)")

Saved YT_merged_structure_annotated.csv (128 rows)
Saved IG_merged_structure_annotated.csv (681 rows)
Saved X_merged_structure_annotated.csv (634 rows)
Saved TT_merged_structure_annotated.csv (283 rows)
Saved YT_merged_column_names_annotated.csv (292 rows)
Saved FB_merged_structure_annotated.csv (5691 rows)
